In [1]:
import pickle
import numpy as np

import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator
import pandas as pd
from IPython.display import display
import scipy
from preprocess import preProcessData

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open("bend_visualization_data_all.pkl", "rb") as f:
    data = pickle.load(f)

pcd_points = data["pcd_points"]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_points)
bend_data = data["bend_data"]
all_normals = data["all_normals"]
segment_indices = data["segment_indices"]
segment_models = data["segment_models"]

In [ ]:
class ArcLengthCalculator:
    def __init__(self, pcd, bend_id, intersection_line, bend_edges, bend_cluster_points, cluster_derivatives, all_normals, segment_indices, intersection_step_size=0.1, derivative_step_size=0.01, radius=0.5):
        self.pcd = pcd
        self.points = np.asarray(pcd.points)

        if len(pcd.points) != len(all_normals):
            raise ValueError(f"Mismatch: {len(pcd.points)} points in PCD, but {len(all_normals)} normals loaded. "
                     "Make sure the point cloud is preprocessed the same way.")
        else:
            print(f"Loaded {len(all_normals)} normals for {len(pcd.points)} points in PCD. Proceeding.")
        
        self.bend_id = bend_ids
        self.intersection_line = intersection_line
        self.bend_cluster_points = bend_cluster_points
        self.bend_cluster_derivatives = cluster_derivatives
        self.bend_cluster_kdtree = cKDTree(self.bend_cluster_points)
        self.pcd_kdtree = cKDTree(self.points)
        self.start_point, self.end_point = map(np.array, bend_edges)
        self.intersection_step_size = intersection_step_size
        self.derivative_step_size = derivative_step_size
        self.all_normals = all_normals
        self.segment_indices = segment_indices
        self.radius = 1.5
        self.intersection_it = 0
        self.derivative_it = 0
        self.is_moving_forward = True

        # self.pcd.paint_uniform_color([0.6, 0.6, 0.6])
        # self.vis = o3d.visualization.Visualizer()
        # self.vis.create_window("ArcLengthCalculator")
        # self.vis.add_geometry(self.pcd)

        self.plane_dominant_bins = {}
        self.plane_dominant_bins[0] = self.get_dominant_bin_ranges(
                self.all_normals[self.segment_indices[0]],
                bin_size=0.5
            )
        
        self.plane_dominant_bins[bend_id] = self.get_dominant_bin_ranges(
                self.all_normals[self.segment_indices[bend_id]],
                bin_size=0.5
            )

    def plot_histogram_with_dominant_bins(self, angles, bin_edges, dominant_ranges, title, color):
        import matplotlib.pyplot as plt
        counts, edges, bars = plt.hist(angles, bins=bin_edges, color=color, edgecolor='black')
        # for i, bar in enumerate(bars):
        #     bin_start = edges[i]
        #     bin_end = edges[i+1]
        #     for dom_start, dom_end in dominant_ranges:
        #         if np.isclose(bin_start, dom_start, atol=1e-6) and np.isclose(bin_end, dom_end, atol=1e-6):
        #             bar.set_color('green')
        plt.title(title)
        plt.tight_layout()
        plt.show()

    def get_dominant_bin_ranges(self, normals, top_k=12, bin_size=0.4):
        dominant_bins = {}

        # Azimuth
        azimuth = np.degrees(np.arctan2(normals[:, 1], normals[:, 0]))
        az_hist, az_edges = np.histogram(azimuth, bins=np.arange(-180, 180 + bin_size, bin_size))
        top_az_indices = np.argsort(az_hist)[-top_k:]

        az_ranges = []
        for idx in top_az_indices:
            az_ranges.append((az_edges[idx], az_edges[idx + 1]))

        # Zenith
        zenith = np.degrees(np.arccos(np.clip(normals[:, 2], -1, 1)))
        zen_hist, zen_edges = np.histogram(zenith, bins=np.arange(0, 180 + bin_size, bin_size))
        top_zen_indices = np.argsort(zen_hist)[-top_k:]

        zen_ranges = []
        for idx in top_zen_indices:
            zen_ranges.append((zen_edges[idx], zen_edges[idx + 1]))

        dominant_bins["azimuth"] = az_ranges
        dominant_bins["zenith"] = zen_ranges

        # # Optional: Visualize
        # self.plot_histogram_with_dominant_bins(azimuth, az_edges, az_ranges, "Azimuth Histogram", color='skyblue')
        # self.plot_histogram_with_dominant_bins(zenith, zen_edges, zen_ranges, "Zenith Histogram", color='salmon')

        return dominant_bins

    def is_normal_in_any_bin(self, normal, bin_ranges, mode):
        if mode == "azimuth":
            angle = (np.degrees(np.arctan2(normal[1], normal[0])) + 360) % 360
        elif mode == "zenith":
            angle = np.degrees(np.arccos(np.clip(normal[2], -1, 1)))
        else:
            raise ValueError("Invalid mode: choose 'azimuth' or 'zenith'")
        
        #print(f"Angle: {angle} degrees")
        #print(f"Bin ranges: {bin_ranges}")
        for start, end in bin_ranges:
            start = (start + 360) % 360
            end = (end + 360) % 360
            if self.angle_in_range(angle, start, end):
                return True
        return False
    
    def angle_in_range(self, angle, start, end):
        """Checks if angle (in degrees) is inside [start, end], handling wrap-around."""
        angle = (angle + 360) % 360
        start = (start + 360) % 360
        end = (end + 360) % 360
        if start <= end:
            return start <= angle <= end
        else:
            return angle >= start or angle <= end
    
    def check_plane_alignment(self, normal):
        result = {}

        zenith_angle = np.degrees(np.arccos(np.clip(normal[2], -1, 1)))

        for plane_id in [0, self.bend_id]:
            bins = self.plane_dominant_bins[plane_id]
            az_bins = bins.get("azimuth", [])
            zen_bins = bins.get("zenith", [])

            # Always check zenith
            zen_match = self.is_normal_in_any_bin(normal, zen_bins, "zenith") if zen_bins else True

            # Only check azimuth if zenith is not ~0 or ~180
            if 2 < zenith_angle < 178:
                az_match = self.is_normal_in_any_bin(normal, az_bins, "azimuth") if az_bins else True
            else:
                az_match = True  # skip azimuth check for vertical normals

            result[plane_id] = az_match and zen_match

        return result
    
    def find_hemisphere_normal(self, neighborhood_points):
        if neighborhood_points.shape[0] < 3:
            print("[WARN] Too few points for PCA")
            return None

        if np.allclose(np.var(neighborhood_points, axis=0), 0):
            print("[WARN] Degenerate neighborhood (no variation)")
            return None

        mean = np.mean(neighborhood_points, axis=0)
        centered = neighborhood_points - mean

        try:
            cov = np.cov(centered.T)
            eig_vals, eig_vecs = np.linalg.eigh(cov)
            eig_vecs = eig_vecs[:, np.argsort(eig_vals)]
            return eig_vecs[:, 0]
        except np.linalg.LinAlgError as e:
            print(f"[WARN] PCA failed due to LinAlgError: {e}")
            return None
        except Exception as e:
            print(f"[WARN] Unexpected error during PCA: {e}")
            return None

    
    def align_normals(self, reference_normal, neighbor_directions):
        aligned_normals = np.array(neighbor_directions)
        
        # Check dot product: If negative, flip the normal
        for i in range(len(aligned_normals)):
            if np.dot(reference_normal, aligned_normals[i]) < 0:
                aligned_normals[i] = -aligned_normals[i]

        return aligned_normals

    def compute_piecewise_arc_length(self, points):
        return np.sum(np.linalg.norm(np.diff(points, axis=0), axis=1))

    def hug_surface(self, last_bend_point, direction_vector): 
        neighbor_indices = self.bend_cluster_kdtree.query(last_bend_point, k=15)[1]
        neighbor_points = self.bend_cluster_points[neighbor_indices]

        if neighbor_points.shape[0] < 3 or np.allclose(np.var(neighbor_points, axis=0), 0):
            return last_bend_point  # Return as-is if too few or degenerate

        try:
            mean = np.mean(neighbor_points, axis=0)
            centered = neighbor_points - mean
            cov = np.cov(centered.T)
            eig_vals, eig_vecs = np.linalg.eigh(cov)
            eig_vecs = eig_vecs[:, np.argsort(eig_vals)]
            pca_normal = eig_vecs[:, 0]
        except np.linalg.LinAlgError as e:
            print(f"[WARN] PCA in hug_surface failed: {e}")
            return last_bend_point
        except Exception as e:
            print(f"[WARN] Unexpected error in hug_surface: {e}")
            return last_bend_point

        # Flip if normal is facing wrong way
        vec_to_point = last_bend_point - mean
        if np.dot(vec_to_point, pca_normal) < 0:
            pca_normal = -pca_normal

        # Project + restrict correction to perpendicular
        distance = np.dot(vec_to_point, pca_normal)
        projected_point = last_bend_point - distance * pca_normal
        correction_vec = projected_point - last_bend_point
        perp_component = correction_vec - np.dot(correction_vec, direction_vector) * direction_vector
        final_point = last_bend_point + perp_component

        return final_point

    
    def find_bend_end(self, bend_point, derivate_direction):
        initial_direction = derivate_direction.copy()
        path_points = [bend_point.copy()]
        i=0
        projected_bend_point = bend_point.copy()
        total_arc_length = 0
        while True:
            neighbor_all_indices = self.pcd_kdtree.query_ball_point(bend_point, self.radius)
            if len(neighbor_all_indices) > 15 and i<160:
                # Separate neighbors by hemisphere
                neighbor_points = self.points[neighbor_all_indices]
                # FULL HEMISPHERE
                # vectors = neighbor_points - bend_point
                # dot_products = vectors @ derivate_direction
                # forward_mask = dot_products > 0
                # forward_indices = np.array(neighbor_all_indices)[forward_mask]
                # #print(f"Forward indices: {forward_indices}")
                # forward_points = self.points[forward_indices]
                # pca_normal = self.find_hemisphere_normal(forward_points)

                # SLAB FORWARD STRUCTURE
                vectors = neighbor_points - bend_point
                dot_products = vectors @ derivate_direction
                epsilon = 0.8
                mask_1 = np.abs(dot_products) < epsilon
                mask_2 = dot_products > 0
                final_mask = mask_1 & mask_2
                #final_mask = mask_1
                forward_indices = np.array(neighbor_all_indices)[final_mask]
                forward_points = self.points[forward_indices]
                if forward_points.shape[0] < 10:
                    total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                    return projected_bend_point, total_arc_length

                try:
                    pca_normal = self.find_hemisphere_normal(forward_points)
                    if pca_normal is None:
                        total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                        return projected_bend_point, total_arc_length
                except np.linalg.LinAlgError:
                    #print(f"[WARN] PCA failed at iteration {i} (bend ID {self.bend_id}) — returning current point.")
                    total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                    return projected_bend_point, total_arc_length

                bend_global_idx = self.pcd_kdtree.query(bend_point)[1]
                original_normal = self.all_normals[bend_global_idx]

                if np.dot(pca_normal, original_normal) < 0:
                    pca_normal = -pca_normal

                alignment = self.check_plane_alignment(pca_normal)
                #print(f"Alignment: Plane 0 = {alignment[0]}, Plane {self.bend_id} = {alignment[self.bend_id]}", flush=True)
                if alignment[0] or alignment[self.bend_id]:
                    #print("Found alignment in direction", derivate_direction)
                    total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                    return projected_bend_point, total_arc_length
                neighbor_bend_indices = self.bend_cluster_kdtree.query_ball_point(bend_point, self.radius)
                if len(neighbor_bend_indices) > 5:
                    neighbor_derivatives = self.bend_cluster_derivatives[neighbor_bend_indices]
                    aligned_neighbor_derivatives = self.align_normals(initial_direction, neighbor_derivatives)
                    derivate_direction = np.mean(aligned_neighbor_derivatives, axis=0)
                    derivate_direction /= np.linalg.norm(derivate_direction)
                bend_point = bend_point + self.derivative_step_size*derivate_direction
                projected_bend_point = self.hug_surface(bend_point, derivate_direction)
                path_points.append(projected_bend_point.copy())
                bend_point = projected_bend_point.copy()
                #print(f'i = {i}')
                i += 1
            else:
                total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                return projected_bend_point, total_arc_length
            
    def calculate_arc_length(self):
        # At the top of the function
        results_table = []
        distances = []
        direction_vector = self.intersection_line[0] / np.linalg.norm(self.intersection_line[0])
        to_end = self.end_point - self.start_point
        if np.dot(direction_vector, to_end) < 0:
            direction_vector *= -1  # Flip if it's pointing toward the wrong side
        
        offset = self.radius
        sampled_point = self.start_point + offset * direction_vector
        end_points_forward = []
        end_points_backward = []
        while np.linalg.norm(sampled_point - self.start_point) < np.linalg.norm(self.end_point - self.start_point) - offset:
            # Sample next point
            sampled_point += self.intersection_step_size * direction_vector
            # Get point closest to the sampled point
            idx = self.bend_cluster_kdtree.query(sampled_point)[1]
            base = self.bend_cluster_points[idx]
            # Get the average derivative of the bend in the neighborhood
            neighbor_bend_indices = self.bend_cluster_kdtree.query_ball_point(base, self.radius)
            neighbor_derivatives = self.bend_cluster_derivatives[neighbor_bend_indices]
            aligned_neighbor_derivatives = self.align_normals(neighbor_derivatives[0], neighbor_derivatives)
            derivate_reference = np.mean(aligned_neighbor_derivatives, axis=0)
            derivate_reference /= np.linalg.norm(derivate_reference)

            # Calculate bend end point in forward direction
            end_point_forward, first_half_distance = self.find_bend_end(base, derivate_reference)
            
            # Calculate bend end point in backward direction
            end_point_backward, second_half_distance = self.find_bend_end(base, -derivate_reference)

            end_points_forward.append(end_point_forward)
            end_points_backward.append(end_point_backward)

            total_distance = first_half_distance + second_half_distance
            distances.append(total_distance)
            results_table.append({
                "Sample Index": len(results_table),
                "Base XYZ": base,
                "Arc Length Forward": first_half_distance,
                "Arc Length Backward": second_half_distance,
                "Total Arc Length": total_distance
            })

        df = pd.DataFrame(results_table)
        if df.empty:
            print(f"[WARN] No samples processed for bend ID {self.bend_id}")
            print(f"start: {self.start_point}, end: {self.end_point}, intersection line length: {np.linalg.norm(self.end_point - self.start_point):.3f}")
            return df
        return df

    # def run(self):
    #     self.vis.run()
    #     self.vis.destroy_window()

In [ ]:
results = []

for bend_id, bend_info in bend_data.items():
    print(f"\nProcessing bend ID: {bend_id}")
    calculator = ArcLengthCalculator(
        pcd=pcd,
        bend_id=bend_id,
        intersection_line=bend_info["intersection_line"],
        bend_edges=bend_info["bend_edges"],
        bend_cluster_points=bend_info["cluster_points"],
        cluster_derivatives=bend_info["cluster_derivatives"],
        all_normals=all_normals,
        segment_indices=segment_indices
        
    )
    df = calculator.calculate_arc_length()
    if df.empty:
        continue
    else:
        mean_arc = df["Total Arc Length"].mean()
        std_arc = df["Total Arc Length"].std()
        print(f"→ Bend {bend_id}: Mean = {mean_arc:.4f} mm | Std = {std_arc:.4f} mm")
        results.append((bend_id, mean_arc, std_arc))

# Print all results together
print("\n=== Arc Length Summary ===")
for bend_id, mean_arc, std_arc in results:
    print(f"Bend {bend_id}: Mean = {mean_arc:.4f} mm | Std = {std_arc:.4f} mm")


Processing bend ID: 1
Loaded 272087 normals for 272087 points in PCD. Proceeding.


/tmp/ipykernel_36891/877155800.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov = np.cov(centered.T)
/home/chris/miniconda3/envs/pointclouds/lib/python3.9/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/chris/miniconda3/envs/pointclouds/lib/python3.9/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/home/chris/miniconda3/envs/pointclouds/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chris/miniconda3/envs/pointclouds/lib/python3.9/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/chris/miniconda3/envs/pointclouds/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(a

In [ ]:
# Visualize scan with arc length labels at anchor points
geometries = [pcd]  # Assume 'pcd' is your full registered scan point cloud
pcd.paint_uniform_color([0.6, 0.6, 0.6])  # Set color for the point cloud
for bend_id, mean_arc, std_arc in results:
    anchor = np.array(bend_data[bend_id]["anchor_points"])

    # Add sphere
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.5)
    sphere.translate(anchor)
    sphere.paint_uniform_color([0, 0, 0])
    geometries.append(sphere)

    try:
        # Create text
        text_mesh = o3d.t.geometry.TriangleMesh.create_text(
            text=f"B{bend_id}: {mean_arc:.2f} mm",
            depth=5
        ).to_legacy()

        # Transform: scale and position
        transform = np.array([
            [0.1, 0,   0,   anchor[0]],
            [0,   0.1, 0,   anchor[1]],
            [0,   0,   0.1, anchor[2] + 4.0],
            [0,   0,   0,   1]
        ])
        text_mesh.transform(transform)
        text_mesh.paint_uniform_color([0.1, 0.1, 0.1])
        geometries.append(text_mesh)

    except Exception as e:
        print(f"[WARN] Could not render text for Bend {bend_id}: {e}")


# Final visualization
o3d.visualization.draw_geometries(geometries)



In [ ]:
angles_rad = util.findAnglesBetweenPlanes(segment_models, 0)
print(f"Angles between planes: {angles_rad}")

angles_deg = {k: np.degrees(v) for k, v in angles_rad.items()}
print("Angles between planes (degrees):")
for k, v in angles_deg.items():
    print(f"Plane {k}: {v:.2f}°")

Angles between planes: {1: 1.6497703298517927, 2: 0.5464586335204168, 3: 1.775494043230384, 4: 0.5485652597677456, 5: 1.1241748000461844, 6: 0.5578631329020617, 7: 0.931523523238162, 8: 0.5643877571392915, 9: 0.743845816336495, 10: 0.6207362852784425, 11: 0.5699079501385356, 12: 0.5015666722499251, 13: 0.5684293741021468, 14: 0.2729811942093728}
Angles between planes (degrees):
Plane 1: 94.52°
Plane 2: 31.31°
Plane 3: 101.73°
Plane 4: 31.43°
Plane 5: 64.41°
Plane 6: 31.96°
Plane 7: 53.37°
Plane 8: 32.34°
Plane 9: 42.62°
Plane 10: 35.57°
Plane 11: 32.65°
Plane 12: 28.74°
Plane 13: 32.57°
Plane 14: 15.64°
